In [13]:
import pandas_datareader as pdr
import pandas as pd
from datetime import datetime, timedelta
import warnings

# Ignore FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

In [14]:
# FRED codes
fred_codes = {
    'Balance sheet assets (as of Wednesday)': 'WTFSRFL',
    'Balance sheet assets (week average)':'WTFSRFA',
    'Holdings of US Treasury securities (as of Wednesday)':'WSHOTSL',
    'Holdings of US Treasury securities (week average)': 'WSHOTSA',
    'Holdings of mortgage-backed securities (as of Wednesday)': 'WSHOMCB',
    'Holdings of mortgage-backed securities (week average)': 'WMBSEC',
    'Discount Window borrowing (as of Wednesday)': 'WLCFLPCL',
    'Discount Window borrowing (week average)': 'WPC',
    'BTFP borrowing (as of Wednesday)': 'H41RESPPALDKNWW',
    'BTFP borrowing (week average)':"H41RESPPALDKXAWNWW"
}

### Code to pull data into dataframe

In [15]:
# Initialize an empty list to store DataFrames
dfs = []

In [16]:
def format_value(value):
    trillion = 1e12 / 1000000
    billion = 1e9 / 1000000
    million = 1e6 / 1000000

    formatted_value = None

    if abs(value) >= trillion:
        formatted_value = f"${abs(value) / trillion:.2f}T"
    elif billion <= abs(value) < trillion:
        formatted_value = f"${abs(value) / billion:.2f}B"
    elif million <= abs(value) < billion:
        formatted_value = f"${abs(value) / million:.2f}M"
    else:
        formatted_value = f"${abs(value):.2f}"

    # Add parentheses for negative values
    if value < 0:
        formatted_value = f"({formatted_value})"

    return formatted_value

In [17]:
# Loop through each FRED code
for report, code in fred_codes.items():
    # Get today's date
    today = datetime.now()

    # Find the Wednesday 53 weeks ago
    last_year_wednesday = today - timedelta(weeks=54)
    while last_year_wednesday.weekday() != 2:  # 2 corresponds to Wednesday
        last_year_wednesday += timedelta(days=1)

    # Get the data for the last 52 weeks
    data_52_weeks = pdr.get_data_fred(code, start=last_year_wednesday, end=today)

    # Column name may vary, get the column dynamically
    value_column = data_52_weeks.columns[0]

    # Sort the data by date in descending order
    data_52_weeks = data_52_weeks.sort_index(ascending=False)

    # Extract most recent, previous, and prior year values
    most_recent_data = data_52_weeks.head(1)
    previous_data = data_52_weeks.iloc[1]
    prior_year_data = data_52_weeks.tail(1)  # Take the last value from the last 52 weeks

    # Extract values
    most_recent_date = most_recent_data.index[0]
    most_recent_value = most_recent_data.iloc[0][value_column]

    previous_date = previous_data.name
    previous_value = previous_data[value_column]

    prior_year_date = prior_year_data.index[0]
    prior_year_value = prior_year_data.iloc[0][value_column]

    # Calculate the changes
    week_over_week_change = most_recent_value - previous_value
    year_over_year_change = most_recent_value - prior_year_value

    # Format values using the custom function
    most_recent_formatted = format_value(most_recent_value)
    previous_formatted = format_value(previous_value)
    prior_year_formatted = format_value(prior_year_value)
    week_over_week_change_formatted = format_value(week_over_week_change)
    year_over_year_change_formatted = format_value(year_over_year_change)

    # Create a DataFrame for the current FRED code and append to the list
    dfs.append(pd.DataFrame({'Report': [report],
                             'Most Recent Date': [most_recent_date],
                             'Most Recent': [most_recent_formatted],
                             'Previous Date': [previous_date],
                             'Previous': [previous_formatted],
                             'Change (w/w)': [week_over_week_change_formatted],
                             'Prior Year Date': [prior_year_date],
                             'Prior Year': [prior_year_formatted],
                             'Change (y/y)': [year_over_year_change_formatted]}))

# Concatenate all DataFrames into one
data = pd.concat(dfs, ignore_index=True)

### Data table

In [18]:
data

,Report,Most Recent Date,Most Recent,Previous Date,Previous,Change (w/w),Prior Year Date,Prior Year,Change (y/y)
0,Balance sheet assets (as of Wednesday),2024-01-17,$7.72T,2024-01-10,$7.74T,($12.91B),2023-01-18,$8.54T,($814.97B)
1,Balance sheet assets (week average),2024-01-17,$7.74T,2024-01-10,$7.73T,$2.48B,2023-01-18,$8.55T,($817.50B)
2,Holdings of US Treasury securities (as of Wedn...,2024-01-17,$4.72T,2024-01-10,$4.75T,($27.49B),2023-01-18,$5.44T,($712.70B)
3,Holdings of US Treasury securities (week average),2024-01-17,$4.74T,2024-01-10,$4.75T,($9.00B),2023-01-18,$5.45T,($707.47B)
4,Holdings of mortgage-backed securities (as of ...,2024-01-17,$2.43T,2024-01-10,$2.43T,($63.00M),2023-01-18,$2.64T,($208.23B)
5,Holdings of mortgage-backed securities (week a...,2024-01-17,$2.43T,2024-01-10,$2.43T,($18.00M),2023-01-18,$2.64T,($209.23B)
6,Discount Window borrowing (as of Wednesday),2024-01-17,$2.29B,2024-01-10,$2.11B,$189.00M,2023-01-18,$4.60B,($2.31B)
7,Discount Window borrowing (week average),2024-01-17,$2.17B,2024-01-10,$2.07B,$98.00M,2023-01-18,$4.15B,($1.98B)
8,BTFP borrowing (as of Wednesday),2024-01-17,$161.50B,2024-01-10,$147.18B,$14.33B,2023-01-18,$0.00,$161.50B
9,BTFP borrowing (week average),2024-01-17,$154.16B,2024-01-10,$144.84B,$9.32B,2023-01-18,$0.00,$154.16B


### Print statement code

In [19]:
def print_balance_sheet_report(data):
    # Extract values for the specific report
    balance_sheet_most_recent = data.loc[data['Report'] == 'Balance sheet assets (as of Wednesday)', 'Most Recent'].values[0]

    # Extract values for Change (w/w) and Change (y/y)
    change_wk = data.loc[data['Report'] == 'Balance sheet assets (week average)', 'Change (w/w)'].values[0]
    change_yr = data.loc[data['Report'] == 'Balance sheet assets (week average)', 'Change (y/y)'].values[0]

    # Add a plus sign for formatted values that don't start and end with parentheses
    change_wk_formatted = "+" + change_wk if not (change_wk.startswith('(') and change_wk.endswith(')')) else change_wk
    change_yr_formatted = "+" + change_yr if not (change_yr.startswith('(') and change_yr.endswith(')')) else change_yr

    # Print statement
    print("Fed reports balance sheet assets of {} on Wednesday, {} w/w and {} y/y".format(balance_sheet_most_recent, change_wk_formatted, change_yr_formatted))

In [20]:
def print_treasury_securities_report(data):
    # Extract values for the specific report
    treasury_most_recent = data.loc[data['Report'] == 'Holdings of US Treasury securities (week average)', 'Most Recent'].values[0]
    treasury_change_wk = data.loc[data['Report'] == 'Holdings of US Treasury securities (week average)', 'Change (w/w)'].values[0]
    treasury_most_recent_date = data.loc[data['Report'] == 'Holdings of US Treasury securities (week average)', 'Most Recent Date'].values[0]
    treasury_change_yr = data.loc[data['Report'] == 'Holdings of US Treasury securities (week average)', 'Change (y/y)'].values[0]

    # Add a plus sign for formatted values that don't start and end with parentheses
    treasury_change_wk_formatted = "+" + treasury_change_wk if not (treasury_change_wk.startswith('(') and treasury_change_wk.endswith(')')) else treasury_change_wk
    treasury_change_yr_formatted = "+" + treasury_change_yr if not (treasury_change_yr.startswith('(') and treasury_change_yr.endswith(')')) else treasury_change_yr

    # Format the date in the format like 17-Jan
    treasury_most_recent_date_formatted = pd.to_datetime(treasury_most_recent_date).strftime('%d-%b')

    # Print statement
    print("Holdings of US Treasury Securities were {} on {}, {} w/w and {} y/y".format(treasury_most_recent, treasury_most_recent_date_formatted, treasury_change_wk_formatted, treasury_change_yr_formatted))


In [21]:
def print_mortgage_backed_securities_report(data):
    # Extract values for the specific report
    mortgage_most_recent = data.loc[data['Report'] == 'Holdings of mortgage-backed securities (week average)', 'Most Recent'].values[0]
    mortgage_change_wk = data.loc[data['Report'] == 'Holdings of mortgage-backed securities (week average)', 'Change (w/w)'].values[0]
    mortgage_most_recent_date = data.loc[data['Report'] == 'Holdings of mortgage-backed securities (week average)', 'Most Recent Date'].values[0]
    mortgage_change_yr = data.loc[data['Report'] == 'Holdings of mortgage-backed securities (week average)', 'Change (y/y)'].values[0]

    # Add a plus sign for formatted values that don't start and end with parentheses
    mortgage_change_wk_formatted = "+" + mortgage_change_wk if not (mortgage_change_wk.startswith('(') and mortgage_change_wk.endswith(')')) else mortgage_change_wk
    mortgage_change_yr_formatted = "+" + mortgage_change_yr if not (mortgage_change_yr.startswith('(') and mortgage_change_yr.endswith(')')) else mortgage_change_yr

    # Format the date in the format like 17-Jan
    mortgage_most_recent_date_formatted = pd.to_datetime(mortgage_most_recent_date).strftime('%d-%b')

    # Print statement
    print("Holdings of mortgage-backed securities were {} on {}, {} w/w and {} y/y".format(mortgage_most_recent, mortgage_most_recent_date_formatted, mortgage_change_wk_formatted, mortgage_change_yr_formatted))

In [22]:
def print_btfp_borrowing_report(data):
    # Extract values for the specific report
    btfp_most_recent = data.loc[data['Report'] == 'BTFP borrowing (week average)', 'Most Recent'].values[0]
    btfp_change_wk = data.loc[data['Report'] == 'BTFP borrowing (week average)', 'Change (w/w)'].values[0]
    btfp_prior_week = data.loc[data['Report'] == 'BTFP borrowing (week average)', 'Previous'].values[0]

    # Add a plus sign for formatted values that don't start and end with parentheses
    btfp_change_wk_formatted = "+" + btfp_change_wk if not (btfp_change_wk.startswith('(') and btfp_change_wk.endswith(')')) else btfp_change_wk

    # Print statement
    print("BTFP borrowing was {} vs {} in prior week.".format(btfp_most_recent, btfp_prior_week))

# Print statements

In [23]:
print_balance_sheet_report(data)
print_treasury_securities_report(data)
print_mortgage_backed_securities_report(data)
print_btfp_borrowing_report(data)

Fed reports balance sheet assets of $7.72T on Wednesday, +$2.48B w/w and ($817.50B) y/y
Holdings of US Treasury Securities were $4.74T on 17-Jan, ($9.00B) w/w and ($707.47B) y/y
Holdings of mortgage-backed securities were $2.43T on 17-Jan, ($18.00M) w/w and ($209.23B) y/y
BTFP borrowing was $154.16B vs $144.84B in prior week.
